# クイックスタート2

## 概要：Azure AI Foundry SDK を使用して Python で簡単なチャット アプリを構築する (2)

このクイックスタートは、クイックスタート1に続いて、Azure AI Foundry SDK を使用してローカル開発環境を設定する方法について説明します。

このクイックスタートは、2025年2月時点での Microsoft の Azure AI Foundry の[ドキュメント](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-code)に記載されている内容になります。

※仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。


1. クイックスタート1で使ったコードを使います。
  - `AIProjectClient` の `from_connection_string()` を使って、Azure AI Foundry のプロジェクトに接続します。
  - `get_chat_completions_client()` を使って、チャットクライアント (`ChatCompletionsClient`) を作成します。
  - システムプロンプトと質問を組み合わせて、指定した LLM モデルのチャットクライアントに問い合わせ (`complete()`) を行います。

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.inference.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')

project = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=DefaultAzureCredential()
)

chat = project.inference.get_chat_completions_client()

2. 推論APIに問い合わせるときに利用するシステムプロンプトを、プロンプトテンプレート文字列として定義します。プロンプトテンプレートには可変の項目を含めることができ、動的なプロンプトを構築できます。このプログラムでは、プロンプトテンプレートを作成して、チャットクライアントに渡します。

In [ ]:
prompt_template_string="""
  system:
  あなたは 2350 年のテクノ パンク ロッカーのように話す AI アシスタントです。クールに、でもクール過ぎないようにしましょう。わかりましたか? ユーザーをファーストネームで呼び、ラストネームをダジャレにしてみてください。

  ユーザーのファーストネームは {{first_name}}、ラストネームは {{last_name}} です
"""

3. 推論APIを呼び出して回答結果を得る関数を定義します。`PromptTemple` の `from_string()` API を使ってプロンプトテンプレートを作成します。この関数では、プロンプトテンプレートの可変項目をコンテキストとして受け取って、`create_message()` APIを使って、チャットクライアントに渡すシステムプロンプトを作成します。システムプロンプトと質問を組み合わせて、チャットクライアントに渡して問い合わせ (`ChatCompletionsClient complete()`) を行います。

In [ ]:
def get_chat_response(messages, context):
    # create a prompt template from an inline string (using mustache syntax)
    prompt_template = PromptTemplate.from_string(prompt_template_string)

    # generate system message from the template, passing in the context as variables
    system_message = prompt_template.create_messages(data=context)

    # add the prompt messages to the user messages
    return chat.complete(
        model=model_name_string,
        messages=system_message + messages,
        temperature=1,
        frequency_penalty=0.5,
        presence_penalty=0.5,
    )

4. 推論APIに問い合わせする質問を定義します。

In [ ]:
query = "世界で一番おいしい食べ物がある都市はどこですか?"
context={"first_name": "一郎", "last_name": "佐藤"}

In [ ]:
query = "世界で一番綺麗な場所がある都市はどこですか?"
context={"first_name": "五郎", "last_name": "田中"}

5. `__main__` 関数を呼び出し、アプリケーションを実行します。

In [ ]:
if __name__ == "__main__":
    response = get_chat_response(
        messages=[{"role": "user", "content": query}],
        context=context,
    )
    print(response.choices[0].message.content)